In [1]:
import sqlite3
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

# === 1. ЗАГРУЗКА ДАННЫХ ===
db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day.db')

# Читаем данные из SQLite
with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        "SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME FROM Futures",
        conn
    )

# === 2. ФУНКЦИЯ КОДИРОВАНИЯ СВЕЧЕЙ (ЛИХОВИДОВ) ===
def encode_candle(row):
    open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']

    if close > open_:
        direction = 1  # Бычья свеча
    elif close < open_:
        direction = 0  # Медвежья свеча
    else:
        direction = 2  # Дожи

    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)

    def classify_shadow(shadow, body):
        if shadow < 0.1 * body:
            return 0  
        elif shadow < 0.5 * body:
            return 1  
        else:
            return 2  

    upper_code = classify_shadow(upper_shadow, body)
    lower_code = classify_shadow(lower_shadow, body)

    return f"{direction}{upper_code}{lower_code}"

df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# === 3. ПОДГОТОВКА ДАННЫХ ДЛЯ НЕЙРОСЕТИ ===
# Преобразуем свечные коды в числовой формат (список уникальных кодов)
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

# Гиперпараметры
window_size = 20  # Длина последовательности
predict_offset = 1  # Предсказываем на 1 день вперед

# Создание входных последовательностей
X, y = [], []
for i in range(len(df_fut) - window_size - predict_offset):
    X.append(df_fut['CANDLE_INT'].iloc[i:i+window_size].values)
    y.append(1 if df_fut['CLOSE'].iloc[i+window_size+predict_offset] > df_fut['CLOSE'].iloc[i+window_size] else 0)

X, y = np.array(X), np.array(y)

# Разделение на train/test
split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# === 4. СОЗДАНИЕ DATASET и DATALOADER ===
class CandlestickDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)  # Для Embedding нужен long
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CandlestickDataset(X_train, y_train)
test_dataset = CandlestickDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# === 5. СОЗДАНИЕ НЕЙРОСЕТИ (LSTM) ===
class CandleLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(CandleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Берем последний временной шаг
        return self.sigmoid(x)

# === 6. ОБУЧЕНИЕ МОДЕЛИ ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CandleLSTM(vocab_size=len(unique_codes), embedding_dim=8, hidden_dim=32, output_dim=1).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Тренировка модели
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# === 7. ТЕСТИРОВАНИЕ ===
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = model(X_batch).squeeze().round()
        correct += (y_pred == y_batch).sum().item()
        total += y_batch.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.2%}")


Epoch 1/20, Loss: 0.6930
Epoch 2/20, Loss: 0.6915
Epoch 3/20, Loss: 0.6912
Epoch 4/20, Loss: 0.6906
Epoch 5/20, Loss: 0.6903
Epoch 6/20, Loss: 0.6901
Epoch 7/20, Loss: 0.6892
Epoch 8/20, Loss: 0.6882
Epoch 9/20, Loss: 0.6873
Epoch 10/20, Loss: 0.6865
Epoch 11/20, Loss: 0.6852
Epoch 12/20, Loss: 0.6828
Epoch 13/20, Loss: 0.6807
Epoch 14/20, Loss: 0.6776
Epoch 15/20, Loss: 0.6755
Epoch 16/20, Loss: 0.6706
Epoch 17/20, Loss: 0.6673
Epoch 18/20, Loss: 0.6636
Epoch 19/20, Loss: 0.6592
Epoch 20/20, Loss: 0.6565
Test Accuracy: 54.06%
